In [1]:
# IMPORTANT ! Note that an additional folder ('Models') needs to be created

import os
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer
import xgboost as xgb
import time
import pickle

In [2]:
# bed and bigwig files contain signals of all chromosomes (including sex chromosomes).
# Training and validation split based on chromosomes has been done for you.
# However, you can resplit the data in any way you want.

# Path for datasets
path_cwd = os.getcwd()
path_data = path_cwd+"/ML4G_Project_1_Data"

# Metadata for genes of cell lines X1 and X2
train_info_X1_path = path_data+"/CAGE-train/CAGE-train/X1_train_info.tsv"
train_info_X2_path = path_data+"/CAGE-train/CAGE-train/X2_train_info.tsv"
val_info_X1_path = path_data+"/CAGE-train/CAGE-train/X1_val_info.tsv"
val_info_X2_path = path_data+"/CAGE-train/CAGE-train/X2_val_info.tsv"

# Gene expression values for cell lines X1 and X2
train_y_X1_path = path_data+"/CAGE-train/CAGE-train/X1_train_y.tsv"
train_y_X2_path = path_data+"/CAGE-train/CAGE-train/X2_train_y.tsv"
val_y_X1_path = path_data+"/CAGE-train/CAGE-train/X1_val_y.tsv"
val_y_X2_path = path_data+"/CAGE-train/CAGE-train/X2_val_y.tsv"

# DNase and histone modification data for cell lines X1, X2 and X3
bed_files_X1 = ["/DNase-bed/X1.bed",
                "/H3K4me1-bed/X1.bed",
                "/H3K4me3-bed/X1.bed",
                "/H3K9me3-bed/X1.bed",
                "/H3K27ac-bed/X1.bed",
                "/H3K27me3-bed/X1.bed",
                "/H3K36me3-bed/X1.bed"]
bed_file_paths_X1 = [path_data+file for file in bed_files_X1]

bed_files_X2 = ["/DNase-bed/X2.bed",
                "/H3K4me1-bed/X2.bed",
                "/H3K4me3-bed/X2.bed",
                "/H3K9me3-bed/X2.bed",
                "/H3K27ac-bed/X2.bed",
                "/H3K27me3-bed/X2.bed",
                "/H3K36me3-bed/X2.bed"]
bed_file_paths_X2 = [path_data+file for file in bed_files_X1]

bed_files_X3 = ["/DNase-bed/X3.bed",
                "/H3K4me1-bed/X3.bed",
                "/H3K4me3-bed/X3.bed",
                "/H3K9me3-bed/X3.bed",
                "/H3K27ac-bed/X3.bed",
                "/H3K27me3-bed/X3.bed",
                "/H3K36me3-bed/X3.bed"]
bed_file_paths_X3 = [path_data+file for file in bed_files_X1]

In [3]:
### FUNCTION FOR EXTRACTION OF FEATURES
def extract_features(bed_path, info_path, max_distance, resolution, stride,verbose=1):
    """
    Function extracting binary features from bed datasets
    :param bed_path: path to bed file of interest
    :param info_path: path to info file of interest
    :param max_distance: maximal distance from TSS that should be considered
    :param resolution: window size of aggregation for dimensionality reduction
    :param stride: stride for dimensionality reduction
    :param verbose: binary feature for printing progress
    :return: pandas df of type int8 containing binary features
    """

    # Load data
    df_info = pd.read_csv(info_path, sep='\t', usecols=[0, 1, 4])
    df_peak_data = pd.read_csv(bed_path, sep='\t', usecols=[0, 1, 2], names=["chromosome", "peak_start", "peak_end"])

    # Get genes and initialize features df with False as entries
    df_features = pd.DataFrame(data=0, columns=[i - max_distance - 1 for i in range(1, 2 * (max_distance + 1))],
                               index=df_info["gene_name"], dtype="int8")

    # Fill df according to info data
    for i in df_info.index:
        gene = df_info["gene_name"][i]
        tss = df_info["TSS_start"][i]
        chromosome = df_info["chr"][i]
        tss_l = tss - max_distance
        tss_r = tss + max_distance

        # Print progress
        if verbose:
            if i == 0:
                print("Start preprocessing of:", "\n" +
                      "Dataset:", bed_path, "\n" +
                      "Infoset:", info_path)
            if i == df_info.index[-1]:
                print("Done!" + "\n" + "-----------------------------------")

        # Find relevant peaks
        peaks = df_peak_data.loc[(df_peak_data["peak_start"] < tss_r) &
                                 (df_peak_data["peak_end"] > tss_l)]

        # Fill features dataset
        for j in range(peaks.shape[0]):
            # Make sure that peak is on the same chromosome
            if peaks["chromosome"].iloc[j] != chromosome: continue

            # Get peak boundaries
            peak_l = peaks["peak_start"].iloc[j]
            peak_r = peaks["peak_end"].iloc[j]

            # Consider possible cases
            if (peak_l >= tss_l) and (peak_r <= tss_r):
                df_features.loc[[gene], peak_l - tss: peak_r - tss] = 1

            elif (peak_l <= tss_r) and (peak_r >= tss_r):
                df_features.loc[[gene], peak_l - tss: tss_r - tss] = 1

            elif (peak_l <= tss_l) and (peak_r <= tss_r):
                df_features.loc[[gene], tss_l - tss: peak_r - tss] = 1

            elif (peak_l <= tss_l) and (peak_r <= tss_r):
                df_features.loc[[gene], tss_l - tss: tss_r - tss] = 1

    # Introduce resolution (rather inefficient...)
    df_features = df_features.rolling(window=resolution,
                                      axis=1,
                                      step=stride,
                                      min_periods=1).mean()

    df_features[df_features > 0.5] = 1
    df_features[df_features <= 0.5] = 0

    return df_features.astype("int8")


In [4]:
### FUNCTION FOR CREATING TRAINING DATASET
def create_set(bed_paths, df_info, max_distance, resolution, stride,verbose=1):
    """
    Create training dataset
    :param bed_paths:
    :param df_info:
    :param max_distance:
    :param resolution:
    :param stride:
    :return:
    """

    df_train = pd.concat([extract_features(path,df_info, max_distance, resolution, stride,verbose) for path in bed_paths], axis=1)
    df_train.columns = [i for i in range(df_train.columns.size)]

    return df_train

def create_set_np(bed_paths, df_info, max_distance, resolution, stride,verbose=0):
    '''
    Equivalent to function above but using numpy arrays to gain efficiency in memory and time
    :param bed_paths:
    :param df_info:
    :param max_distance:
    :param resolution:
    :param stride:
    :param verbose:
    :return:
    '''
    for idx,path in enumerate(bed_paths):
        n=len(bed_paths)
        if idx==0:
            temp=extract_features(path,df_info, max_distance, resolution, stride,verbose)
            n_genes, n_timestamps=temp.shape
            features=np.zeros((n_genes, n_timestamps*n))
        else:
            features[:,idx*n_timestamps: (idx+1)*n_timestamps]=extract_features(path,df_info, max_distance, resolution, stride,verbose).to_numpy()
    return features

In [5]:
def score_func(y, y_pred):
    return spearmanr(y,y_pred).statistic

scorer=make_scorer(score_func) #needed to be able to use spearmanr as score function in scikit-learn

def Train_Test_loop(window_size, resolution,stride,inner_params,train_paths,val_paths,test_paths,mod_identifier_path='1to2',verbose=1):
    '''

    :param window_size:
    :param resolution:
    :param stride:
    :param inner_params:
    :param train_paths: look above to see how to define
    :param val_paths:
    :param test_paths:
    :param verbose:
    :return:
    '''
    results = {}
    results['score_test'], results['score_val'], results['time'], results['model'] = [], [], [], []
    for a in inner_params.keys():
        results[a] = []
    best_score = 0
    best_model = None
    file_name = path_cwd+'/Models'
    counter, n_iter = 0, len(window_size)*len(resolution) #to monitor training progress
    outer_params=[window_size, resolution,stride]
    outer_params=[[i] if type(i)!= type([]) else i for i in outer_params]
    for w in outer_params[0]:
        for r in outer_params[1]:
            for s in outer_params[2]:
                counter += 1
                print(f'Iteration {counter} out of {n_iter}\nWINDOW: {w}, RESOLUTION: {r}, STRIDE:{s}')
                start =time.time()
                #creation of datasets
                y_train = pd.read_csv(train_paths[2], delimiter="\t")['gex'].to_numpy()
                X_train = create_set_np(train_paths[0], train_paths[1], w, r, s)
                y_val = pd.read_csv(val_paths[2], delimiter="\t")['gex'].to_numpy()
                X_val = create_set_np(val_paths[0], val_paths[1], w, r, s)
                y_test = pd.read_csv(test_paths[2], delimiter="\t")['gex'].to_numpy()
                X_test = create_set_np(test_paths[0], test_paths[1], w, r, s)
                X_complete_train = np.concatenate([X_train,X_val])
                y_complete_train = np.concatenate([y_train,y_val])
                n_train = X_train.shape[0]
                CV = [([i for i in range(n_train)],[i for i in range(n_train, y_complete_train.shape[0])])]
                end = time.time()
                if verbose:
                    print(f'Number of features: {X_complete_train.shape[1]}')
                    print(f'\nPre-processing ended in: {round(end-start)} seconds')
                #model definition and training
                model = xgb.XGBRegressor(booster='gbtree')
                opt = BayesSearchCV(
                    model,
                    inner_params,
                    scoring = scorer,
                    n_iter = 40,
                    random_state = 7,
                    cv = CV,
                    verbose = 0)
                start = time.time()
                opt.fit(X_complete_train,y_complete_train)
                end = time.time()
                #results update
                score = spearmanr(opt.predict(X_test),y_test).statistic
                if verbose:
                    print(f'Hyperparameter search ended in: {round(end-start)} seconds\n'
                          f'Optimal hyper parameters:{[(a,b) for a,b in opt.best_params_.items()]}\n'
                          f'Score in Validation: {round(opt.best_score_,4)}')
                    print(f'Score in Test Set: {round(score,4)}\n------------------')
                results['score_test']+=[round(score,4)]
                results['score_val']+=[round(opt.best_score_,4)]
                results['time']+=[round(end-start)]
                for a,b in opt.best_params_.items():
                    results[a]+=[b]
                results['model']=opt.best_estimator_
                if score> best_score:
                    best_model_index = (w,r,s)
                    best_score = score
                    pickle.dump(opt,open(path_cwd+'/Models/best_model'+ mod_identifier_path+'.pickle','wb'))
    index = pd.MultiIndex.from_product(outer_params,names=['window_size','resolution','stride'])
    results = pd.DataFrame(results, index=index)
    pickle.dump(results, open(path_cwd + '/Models/results' + mod_identifier_path+ '.pickle', 'wb'))
    return results, best_model_index



In [6]:
window_size=[100,200]
resolution=[10,20]
stride=10


inner_params={
    'n_estimators': Integer(20,150),
    'learning_rate': Real(1e-5,1e-1,prior='log-uniform'),
    'max_depth': Integer(1,10),
    'reg_lambda':Integer(1,100)
}

In [7]:
train_paths=[bed_file_paths_X1,train_info_X1_path, train_y_X1_path]
val_paths=[bed_file_paths_X1,val_info_X1_path, val_y_X1_path]
test_paths=[bed_file_paths_X2,val_info_X2_path, val_y_X2_path]
results,best_model=Train_Test_loop(window_size, resolution,stride,inner_params,train_paths,val_paths,test_paths,mod_identifier_path='1to2',verbose=1)
results

Iteration 1 out of 4
WINDOW: 100, RESOLUTION: 10, STRIDE:10
Number of features: 147

Pre-processing ended in: 65 seconds
Hyperparameter search ended in: 65 seconds
Optimal hyper parameters:[('learning_rate', 1e-05), ('max_depth', 6), ('n_estimators', 147), ('reg_lambda', 1)]
Score in Validation: 0.7684
Score in Test Set: 0.6785
------------------
Iteration 2 out of 4
WINDOW: 100, RESOLUTION: 20, STRIDE:10
Number of features: 147

Pre-processing ended in: 66 seconds


C:\Users\gugli\anaconda3\envs\pytorch\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Hyperparameter search ended in: 61 seconds
Optimal hyper parameters:[('learning_rate', 1e-05), ('max_depth', 5), ('n_estimators', 150), ('reg_lambda', 38)]
Score in Validation: 0.7713
Score in Test Set: 0.6766
------------------
Iteration 3 out of 4
WINDOW: 200, RESOLUTION: 10, STRIDE:10
Number of features: 287

Pre-processing ended in: 74 seconds
Hyperparameter search ended in: 86 seconds
Optimal hyper parameters:[('learning_rate', 0.1), ('max_depth', 4), ('n_estimators', 20), ('reg_lambda', 100)]
Score in Validation: 0.7746
Score in Test Set: 0.6791
------------------
Iteration 4 out of 4
WINDOW: 200, RESOLUTION: 20, STRIDE:10
Number of features: 287

Pre-processing ended in: 72 seconds


C:\Users\gugli\anaconda3\envs\pytorch\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\gugli\anaconda3\envs\pytorch\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Hyperparameter search ended in: 82 seconds
Optimal hyper parameters:[('learning_rate', 1e-05), ('max_depth', 5), ('n_estimators', 20), ('reg_lambda', 100)]
Score in Validation: 0.7748
Score in Test Set: 0.6761
------------------


score_test  score_val  time  \
window_size resolution stride                                
100         10         10          0.6785     0.7684    65   
            20         10          0.6766     0.7713    61   
200         10         10          0.6791     0.7746    86   
            20         10          0.6761     0.7748    82   

                                                                           model  \
window_size resolution stride                                                      
100         10         10      XGBRegressor(base_score=None, booster='gbtree'...   
            20         10      XGBRegressor(base_score=None, booster='gbtree'...   
200         10         10      XGBRegressor(base_score=None, booster='gbtree'...   
            20         10      XGBRegressor(base_score=None, booster='gbtree'...   

                               n_estimators  learning_rate  max_depth  \
window_size resolution stride                                           
100         10         10               147        0.00001          6   
            20         10               150        0.00001          5   
200         10         10                20        0.10000          4   
            20         10                20        0.00001          5   

                               reg_lambda  
window_size resolution stride              
100         10         10               1  
            20         10              38  
200         10         10             100  
            20         10             100

In [8]:
train_paths=[bed_file_paths_X1,train_info_X1_path, train_y_X1_path]
val_paths=[bed_file_paths_X1,val_info_X1_path, val_y_X1_path]
test_paths=[bed_file_paths_X2,val_info_X2_path, val_y_X2_path]


results,best_model=Train_Test_loop(window_size, resolution,stride,inner_params,train_paths,val_paths,test_paths,mod_identifier_path='2to1',verbose=1)
results

Iteration 1 out of 4
WINDOW: 100, RESOLUTION: 10, STRIDE:10
Number of features: 147

Pre-processing ended in: 66 seconds
Hyperparameter search ended in: 66 seconds
Optimal hyper parameters:[('learning_rate', 1e-05), ('max_depth', 6), ('n_estimators', 147), ('reg_lambda', 1)]
Score in Validation: 0.7684
Score in Test Set: 0.6785
------------------
Iteration 2 out of 4
WINDOW: 100, RESOLUTION: 20, STRIDE:10
Number of features: 147

Pre-processing ended in: 76 seconds


C:\Users\gugli\anaconda3\envs\pytorch\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Hyperparameter search ended in: 60 seconds
Optimal hyper parameters:[('learning_rate', 1e-05), ('max_depth', 5), ('n_estimators', 150), ('reg_lambda', 38)]
Score in Validation: 0.7713
Score in Test Set: 0.6766
------------------
Iteration 3 out of 4
WINDOW: 200, RESOLUTION: 10, STRIDE:10
Number of features: 287

Pre-processing ended in: 69 seconds
Hyperparameter search ended in: 77 seconds
Optimal hyper parameters:[('learning_rate', 0.1), ('max_depth', 4), ('n_estimators', 20), ('reg_lambda', 100)]
Score in Validation: 0.7746
Score in Test Set: 0.6791
------------------
Iteration 4 out of 4
WINDOW: 200, RESOLUTION: 20, STRIDE:10
Number of features: 287

Pre-processing ended in: 66 seconds


C:\Users\gugli\anaconda3\envs\pytorch\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\gugli\anaconda3\envs\pytorch\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Hyperparameter search ended in: 77 seconds
Optimal hyper parameters:[('learning_rate', 1e-05), ('max_depth', 5), ('n_estimators', 20), ('reg_lambda', 100)]
Score in Validation: 0.7748
Score in Test Set: 0.6761
------------------


score_test  score_val  time  \
window_size resolution stride                                
100         10         10          0.6785     0.7684    66   
            20         10          0.6766     0.7713    60   
200         10         10          0.6791     0.7746    77   
            20         10          0.6761     0.7748    77   

                                                                           model  \
window_size resolution stride                                                      
100         10         10      XGBRegressor(base_score=None, booster='gbtree'...   
            20         10      XGBRegressor(base_score=None, booster='gbtree'...   
200         10         10      XGBRegressor(base_score=None, booster='gbtree'...   
            20         10      XGBRegressor(base_score=None, booster='gbtree'...   

                               n_estimators  learning_rate  max_depth  \
window_size resolution stride                                           
100         10         10               147        0.00001          6   
            20         10               150        0.00001          5   
200         10         10                20        0.10000          4   
            20         10                20        0.00001          5   

                               reg_lambda  
window_size resolution stride              
100         10         10               1  
            20         10              38  
200         10         10             100  
            20         10             100

In [11]:
#To load the model
mod=pickle.load(open(path_cwd+'/Models/best_model1to2.pickle','rb'))
mod

BayesSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                    17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, ...],
                   [14310, 14311, 14312, 14313, 14314, 14315, 14316, 14317,
                    14318, 14319, 14320, 14321, 14322, 14323, 14324, 14325,
                    14326, 14327, 14328, 14329, 14330, 14331, 14332, 14333,
                    14334, 14335, 14336, 14337, 14338, 14339, ...])],
              estimator=XGBRegressor(base_score=None, booster='gbtree',
                                     c...
              n_iter=40, random_state=7, scoring=make_scorer(score_func),
              search_spaces={'learning_rate': Real(low=1e-05, high=0.1, prior='log-uniform', transform='normalize'),
                             'max_depth': Integer(low=1, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=20, high=150, prior='uniform', transform='normalize'),
                             'reg_lambda': Integer(low=1, high=100, prior='uniform', transform='normalize')})

In [12]:
res=pickle.load(open(path_cwd+'/Models/results1to2.pickle','rb'))
res

score_test  score_val  time  \
window_size resolution stride                                
100         10         10          0.6785     0.7684    65   
            20         10          0.6766     0.7713    61   
200         10         10          0.6791     0.7746    86   
            20         10          0.6761     0.7748    82   

                                                                           model  \
window_size resolution stride                                                      
100         10         10      XGBRegressor(base_score=None, booster='gbtree'...   
            20         10      XGBRegressor(base_score=None, booster='gbtree'...   
200         10         10      XGBRegressor(base_score=None, booster='gbtree'...   
            20         10      XGBRegressor(base_score=None, booster='gbtree'...   

                               n_estimators  learning_rate  max_depth  \
window_size resolution stride                                           
100         10         10               147        0.00001          6   
            20         10               150        0.00001          5   
200         10         10                20        0.10000          4   
            20         10                20        0.00001          5   

                               reg_lambda  
window_size resolution stride              
100         10         10               1  
            20         10              38  
200         10         10             100  
            20         10             100